## Multinomial Variation Autoencoder for Collaborative Filtering

I must admit that when it comes to variational autoencoders (VAEs) I find that there is a "notable" difference between the complexity of the math and that of the code (or maybe is just me that I am not a mathematician). Nonetheless, I think that speaking about VAEs and not mentioning log likelihoods, Evidence Lower Bound (EBLO) or the Kullback–Leibler divergence ($\text{D}_{\text{KL}}$) is almost like "cheating". With that in mind I will try to give some mathematical context to the $\text{Mult-VAE}^{\text{PR}}$ for collaborative filtering and then move to the code, from how one prepares the data to a discussion of the results. Bear in mind that the whole purpose of the math below is to justify the loss function we will be using when training the $\text{Mult-VAE}^{\text{PR}}$ as well as the architecture of the algorithm.

Before diving into the problem scenario and the mathematical formulation, let me describe the notational convention. Following [Liang et al., 2018](https://arxiv.org/pdf/1802.05814.pdf), I will use $u \in \{1,\dots,U\}$ to index users and $i \in \{1,\dots,I\}$ to index items. The user-by-item **binary** interaction matrix (i.e. the click matrix) is $\mathbf{X} \in \mathbb{N}^{U\times I}$ and I will use lower case $\mathbf{x}_u =[X_{u1},\dots,X_{uI}]^\top \in \mathbb{N}^I$ to refer to the click history of an individual user $u$.

### 1.1 Problem scenario

We are given a dataset $\mathbf{X} = \{ {\mathbf{x}_u} \}^{U}_{u=1}$ of user clicks (a more general scenario is described in "[Auto-Encoding Variational Bayes](https://arxiv.org/pdf/1312.6114.pdf)" [2]). Our job is to estimate the parameters of the underlying probability distribution so that we can do inference. In other words, we need to find a statistical model of the data. To do this, we need to maximize the likelihood function so that under the assumed statistical model the observed data is most probable. 

To find the maximum likelihood we normally assume that the statistical model of the data involves some latent variable $\bf{z}$, so that the marginal likelihood can be written as:

$$
p_{\theta}(\mathbf{x}_u) = \int {p_{\theta}(\mathbf{z}_u)p_{\theta}(\mathbf{x}_u \vert \mathbf{z}_u) d\mathbf{z}_u} \hspace{1cm} (1) 
$$

Eq (1) is solvable if we assume that both the prior $p_{\theta}(\mathbf{z}_u)$ and the conditional probability $p_{\theta}(\mathbf{x}_u \vert \mathbf{z}_u)$ come from parametric families of distributions and that their PDFs are differentiable almost everywhere w.r.t. both $\theta$ and $\mathbf{z}_u$. However, for "*moderately*" complicated likelihood functions $p_{\theta}(\mathbf{x}_u \vert \mathbf{z}_u)$, such as a neural network with a nonlinear layer, Eq (1) is not intractable (it is not possible to evaluate of differentiate the marginal likelihood). Furthermore, the true posterior  $p_{\theta}(\mathbf{z}_u \vert \mathbf{x}_u) = p_{\theta}(\mathbf{x}_u \vert \mathbf{z}_u)p_{\theta}(\mathbf{z}_u)/p_{\theta}(\mathbf{x}_u)$ is also intractable, and therefore we cannot use an EM algorithm (since the E-step involves the computation of the true posterior at a given iteration). 

To address these, and some other limitations, [Kingma and Welling 2014](https://arxiv.org/pdf/1312.6114.pdf) proposed a flexible neural network based approach.

### 1.2 Auto-Encoding Variational Bayes

The following Section is both a summary and my understanding of the paper "[Auto-Encoding Variational Bayes](https://arxiv.org/pdf/1312.6114.pdf)" to which I keep referring and that I strongly recommend reading. 

Let me remind you: our goal is to maximize the likelihood, or more conveniently the log likelihood $\log p_{\theta}(\mathbf{X})$, where:

$$
\log p_{\theta}(\mathbf{X}) = \sum_u \log p_{\theta}(\mathbf{x}_u) \hspace{1cm} (2) 
$$

Each term in the summation can be re-written as:

$$
\log p_{\theta}(\mathbf{x}_u) = D_{KL}\left(q_\phi(\textbf{z}_u\vert \textbf{x}_u) \| p_\theta(\textbf{z}_u \vert \textbf{x}_u)\right) + \underbrace{\mathbb{E} \small{ q_{\phi}(\mathbf{z}_u \vert \mathbf{x}_u) } \left[ -\log q_{\phi}(\mathbf{z}_u \vert \mathbf{x_u}) + \log p_{\theta}(\mathbf{x}_u, \mathbf{z}_u) \right]}_{ELBO \mathcal L(\textbf{x}_u, \phi,\theta)} \hspace{1cm} (3)
$$



Where the first elements in the right hand side is the [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) ($\text{D}_{\text{KL}}$) and $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$ is the approximate posterior of the true posterior $p_\theta(\textbf{z}_u \vert \textbf{x}_u)$. Eq (3) is our "*point of entry*" from which we will derive the remaining equations. If you want proof of Eq (3) I would recommend reading [this tutorial](https://vannevar.ece.uw.edu/techsite/papers/documents/UWEETR-2010-0002.pdf) or this ["crazy" post](https://medium.com/@jonathan_hui/machine-learning-summary-proof-terms-8ca7c588905e). 

Moving on, given that $\text{D}_{\text{KL}}$ is non-negative, $\log p_{\theta}(\mathbf{x}_u) \geq  \mathcal L(\textbf{x}_u, \phi,\theta)$ and therefore  $\mathcal L$ is referred as Evidence Lower Bound (ELBO). It is straightforward to understand from Eq (3) that maximizing $\log p_{\theta}(\mathbf{x}_u)$ implies maximizing ELBO $\mathcal L$. If we re-order the terms in that equation, we could also think of the problem as follows: maximizing ELBO $\mathcal L$ implies minimizing $\text{D}_{\text{KL}}$, which makes sense, since $D_{KL}$ measures the dissimilarity between the approximate posterior  $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$ and the true posterior $p_{\theta}(\textbf{z}_u\vert \textbf{x}_u)$. 

ELBO $\mathcal L$ in Eq (3) can also be re-written as:

$$
\mathcal L(\textbf{x}_u, \phi,\theta) =  - D_{KL}\left(q_\phi(\textbf{z}_u\vert \textbf{x}_u) \| p_\theta(\textbf{z}_u \right) + \mathbb{E} \small{ q_{\phi}(\mathbf{z}_u \vert \mathbf{x}_u) } \left[ \log p_{\theta}(\textbf{x}_u\vert \textbf{z}_u) \right] \hspace{1cm} (4)
$$

We can see that Eq (4) involves sampling $\tilde{\mathbf{z}_u} \sim q_{\phi}(\mathbf{z}_u \vert \mathbf{x}_u)$. When sampling is involved, backpropagation is not trivial (how one would take gradients with respect to $\phi$?). To remedy this situation the authors used the so called "*reparameterization trick*". Instead of sampling from the approximate postertior $q_{\phi}(\mathbf{z}_u \vert \mathbf{x}_u)$, the authors used a differentiable transformation $g_{\phi}(\mathbf{\epsilon}, \mathbf{x}_u)$ of a noise variable $\epsilon$, such that:

$$
\tilde{\mathbf{z}_u} = g_{\phi}(\mathbf{\epsilon}, \mathbf{x}_u) \hspace{1cm} with \hspace{1cm} \mathbf{\epsilon} \sim p(\epsilon) \hspace{1cm} (5)
$$

where $p(\epsilon)$ can be, for example, a variable sampled from a random normal distribution (see Section 1.3 for the selection of $g_{\phi}$ in the particular case of the $\text{Mult-VAE}^{\text{PR}}$). With these formulation, one can use Monte Carlo estimates of expectations of some function $f(\mathbf{z})$ with respect to $q_{\phi}(\mathbf{z}_u \vert \mathbf{x}_u)$ such that: 

$$
\mathbb{E} \small{ q_{\phi}(\mathbf{z}_u \vert \mathbf{x}_u) }\left[ f(\mathbf{z}_u) \right] =  \mathbb{E} \small{ q_{\phi}(\mathbf{z}_u \vert \mathbf{x}_u) }\left[ f(g_{\phi}(\mathbf{\epsilon}, \mathbf{x}_u)) \right] \simeq \frac{1}{L} \sum_{l=1}^{L} f(g_{\phi}(\mathbf{\epsilon}^l), \mathbf{x}_u) 
\\
\text{where} \hspace{1cm} \mathbf{\epsilon}^l \sim p(\epsilon) \hspace{1cm} (6)
$$

Replacing the second term in Eq (4) with the result in Eq (6), we see that the ELBO $\mathcal L$ can be approximated by what Kingma and Welling called generic Stochastic Gradient Variational Bayes (SGVB) estimator $\tilde{\mathcal L}(\textbf{x}_u, \phi,\theta) \simeq \mathcal L(\textbf{x}_u, \phi,\theta)$:

$$
\tilde{\mathcal L}(\mathbf{x}_u, \phi,\theta) = - D_{KL}\left(q_\phi(\textbf{z}_u\vert \textbf{x}_u) \| p_\theta(\textbf{z}_u \right) + \frac{1}{L} \sum_{l=1}^{L} \log p_{\theta}(\mathbf{x}_u \vert \mathbf{z}^l_u) \\
\text{where} \hspace{1cm} \mathbf{z}^l_u = g_{\phi}(\epsilon^l_u,  \mathbf{x}_u) \hspace{1cm} \text{and} \hspace{1cm} \epsilon^l \sim p(\epsilon) \hspace{1cm} (7)
$$

Before we move on to Multinomial variational autoencoders, there is one more important result to discuss. When running a practical application, we will be using minibatches, so that: 

$$
\mathcal L(\mathbf{\text{X}}^M, \phi,\theta) \simeq \tilde{\mathcal L}^{M}(\mathbf{\text{X}}^M, \phi,\theta) = \frac{1}{M} \sum_{u=1}^{M} \tilde{\mathcal L}(\mathbf{x}_u, \phi,\theta) \hspace{1cm} (8)
$$

where $\mathbf{X}^M = \{\mathbf{x}_u \}_{u=1}^M$ is a minibatch of M users. In their experiments the authors found that the number of samples $L$ can be set to 1 as long as the minibatch size was large enough, e.g. $M$ = 100. With this in mind, as long as our batch sizes are of 100 or more, Eq (7) can be re-written as:

$$
\mathcal L(\mathbf{\text{X}}^M, \phi,\theta) \simeq \frac{1}{M} \sum_{u=1}^{M} - D_{KL}\left(q_\phi(\textbf{z}_u\vert \textbf{x}_u) \| p_\theta(\textbf{z}_u \right) + \log  p_{\theta}(\mathbf{x}_u \vert \mathbf{z}^s_u) \hspace{1cm} (9)
$$

Note that $\mathbf{z}^s_u$ signifies that $\mathbf{z}_u$ stil needs to be sampled once from $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$, but using the reparameterization trick this will be rather easy, as we will see in the next section. Finally, now that we have a "nice looking" mathematical expression, this is how Auto-Encoding Variational Bayes works: 

1. Select a prior for latent representation of $\textbf{x}_u$, $p_{\theta}(\textbf{z}_u)$
2. Use a neural network to parameterize the distribution $p_{\theta}(\textbf{x}_u\vert \textbf{z}_u)$. Because this part of the model maps the latent variable/representation $\textbf{z}_u$ to the observed data $\textbf{x}_u$, it is referred as a "*decoder*" network. 
3. Rather than explicitly calculating the intractable posterior $p_{\theta}(\textbf{z}_u\vert \textbf{x}_u)$, use another another neural network to parameterize the distribution $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$ as the approximate posterior. Since $q_\phi$ maps the observed data $\textbf{x}_u$ to the latent space of $\textbf{z}_u$'s, is referred as the "*encoder*" network.
4. maxmize ELBO $\mathcal{L}$ in Eq (9) using Stochastic Gradient Descent or any of its cousins

### 1.3 The $\text{Mult-VAE}^{\text{PR}}$

The set up used by [Liang and co-authors](https://arxiv.org/pdf/1802.05814.pdf) is the following: for each user $u$, the latent representation $\textbf{z}_u$ is assumed to be drawn from a standard Gaussian prior $p(\textbf{z}_u) \sim \mathcal N(0, I)$. Such representation is then transformed by a multi-layer perceptron (MLP), and the output is normalized via a Softmax function to produce a probability distribution over all items **$I$**, $\pi(\mathbf{z}_u) = Softmax(MLP(\mathbf{z}_u))$. Then, the click history of user $u$ is assumed to be drawn from a Multinomial distribution with probability $\pi(\mathbf{z}_u)$: 

$$
\textbf{x}_u \sim \text{Mult}(N_u, \pi(\mathbf{z}_u)) \hspace{1cm} (10)
$$

where $N_u = \sum_i x_{ui}$ is the total number of clicks for user $u$. In this set up, the log-likelihood of the click history $\mathbf{x}_u$ conditioned to the latent representation $\mathbf{z}_u$ is simply:

$$
\begin{equation*}
\log(p_{\theta}(\textbf{x}_u\vert \textbf{z}_u)) = \mathbf{x}_u \log(\pi(\mathbf{z}_u)) \hspace{1cm} (11)
\end{equation*}
$$

The posterior $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$ is also chosen to be a standard Gaussian $q_\phi(\textbf{z}_u\vert \textbf{x}_u) \sim \mathcal N(\mu_\phi(\textbf{x}_u), \sigma_\phi(\textbf{x}_u) I)$ where $\mu_\phi(\textbf{x}_u)$ and  $\sigma_\phi(\textbf{x}_u)$ are functions implemented as neural networks. Then, we use the reparameterization trick and chose $g_{\phi}(\mathbf{\epsilon}, \mathbf{x}_u) = \mu(\textbf{x}_u) + \sigma(\textbf{x}_u) \cdot \epsilon$, where $\epsilon \sim \mathcal{N}(0,I)$. This way $\mathbf{z}^s_u = \mu(\textbf{x}_u) + \sigma(\textbf{x}_u) \cdot \epsilon$ where we sample directly $\epsilon$. 

At this stage we have defined the Gaussian prior, the Gaussian approximate posterior and our sampled latent representation. We are finally ready to write the loss function that we will minimize when training the Mult-VAE:

$$
Loss = -\frac{1}{M} \sum_{u=1}^{M} \left[ \mathbf{x}_u \log(\pi(\mathbf{z}_u)) + \frac{\beta}{2}  \sum_j ( 1 + \log(\sigma_{uj}^2) - \mu_{uj}^2 - \sigma_{uj}^2 )  \right] \hspace{1cm} (12)
$$

Note that the expression above is the negative ELBO $\mathcal L$ (maximizing $\mathcal L$ is equivalent to minimize -$\mathcal L$) with a multiplicative factor $\beta$ applied to the $D_{KL}$. For the behind the $D_{KL}$ expression given this set up have a look here [here](https://stats.stackexchange.com/questions/318748/deriving-the-kl-divergence-loss-for-vaes). 

Let me just comment on that $\beta$. Looking at the loss function in Eq (12) within the context of VAEs, we can see that the first term is the reconstruction loss, while the $D_{KL}$ act as a regularizer. With that in mind, Liang et al add a factor $\beta$ to control the strength of the regularization, and propose $\beta < 1$. Let's paused for one second and think on what this means. First of all, we are no longer optimizing a lower bound for a given log likelihood. In addition, remember that the $D_{KL}$ divergence measures the similarity between the approximate posterior $q_\phi(\textbf{z}_u\vert \textbf{x}_u)$ and the prior $p_\theta(\textbf{z}_u)$. Therefore, by using $\beta < 1$ we are weakening the influence of the prior constrain $q_\phi(\textbf{z}_u\vert \textbf{x}_u) \approx p_\theta(\textbf{z}_u)$ on the loss. This means that we are less able to generalize to novel user clicks from historical data. However, when building recommendation systems we are often not interested in reproducing precisely click histories (i.e. achieving the best loss) but in making good recommendations (i.e. achieving the best ranking metrics). As the authors show in the [paper](https://arxiv.org/pdf/1802.05814.pdf) (and we will see here later), the best ranking metrics are obtained when using $\beta < 1$ and in consequence they name the algorithm Partially Regularized Multinomial Autoencoder or $\text{Mult-VAE}^{\text{PR}}$.

And..."*that's it!*". After all this we are ready to move to the code. There are two comments to make as we move to the code. I have implemented the Mult-VAE using `Mxnet` and `Pytorch`. Given the fact that these two implementations are nearly identical, I will concentrate here only on the `Mxnet` implementation, please go to the repo if you want to have a look to the `Pytorch` one. Secondly, the paper also includes a Multinomial Denoising Autoencoder, referred as Mult-DAE. However, given its simplicity relative to the formulation of the Mult-VAE discussed here I will not discussed the math of the Mult-DAE and focus simply on the code.

### $\text{Mult-VAE}^{\text{PR}}$, the code

After the explanation in the section above you might expect the code to look rather complex. However, you might feel disappointed/pleased when you see how simple it really is. 

In the [original publications](https://arxiv.org/pdf/1802.05814.pdf) the authors used a one hidden layer MLP as generative model. There they say that deeper architectures do not improve the results. In notebook 04 I will show the results and we will see that they are actually right. I have run over a hundred experiments and I find that the set up described in the paper leads to the best results. With that it mind, let's first have a look the model $ I \rightarrow 600 \rightarrow 200 \rightarrow 600 \rightarrow I$, where $I$ is the total number of items: 

<p align="center">
  <img width="800" src="figures/multvae_arch.png">
</p>

Fig1. 

In code, the model in the figure above is:

In [2]:
from typing import List

from mxnet import autograd
from mxnet.gluon import nn, HybridBlock

class VAEEncoder(HybridBlock):
    def __init__(self, q_dims: List[int], dropout: List[float]):
        super().__init__()

        # last dim multiplied by two for the reparameterization trick
        q_dims_ = q_dims[:-1] + [q_dims[-1] * 2]
        with self.name_scope():
            self.q_layers = nn.HybridSequential(prefix="q_net")
            for p, inp, out in zip(dropout, q_dims_[:-1], q_dims_[1:]):
                self.q_layers.add(nn.Dropout(p))
                self.q_layers.add(nn.Dense(in_units=inp, units=out))

    def hybrid_forward(self, F, X):
        h = F.L2Normalization(X)
        for i, layer in enumerate(self.q_layers):
            h = layer(h)
            if i != len(self.q_layers) - 1:
                h = F.tanh(h)
            else:
                mu, logvar = F.split(h, axis=1, num_outputs=2)
        return mu, logvar

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Before I move on, let me mention (and appreciate) one of the many nice "little" things that `Mxnet`'s `Gluon` has to offer. You will notice the use of `HybridBlock` and the use of the input `F` (the backend) when we define the forward pass, or more precisely, the `hybrid_forward` pass. One could write a full post on the joys of `HybridBlocks` and how nicely and easily the guys that developed `Gluon` brought together the flexibility of imperative frameworks (i.e. `Pytorch`) and the speed of declarative frameworks (i.e. `Tensorflow`) together. If you want to learn the details go [here](https://gluon.mxnet.io/chapter07_distributed-learning/hybridize.html), but believe me, this is **FAST**. 

In [7]:
class Decoder(HybridBlock):
    def __init__(self, p_dims: List[int], dropout: List[float]):
        super().__init__()

        with self.name_scope():
            self.p_layers = nn.HybridSequential(prefix="p_net")
            for p, inp, out in zip(dropout, p_dims[:-1], p_dims[1:]):
                self.p_layers.add(nn.Dropout(p))
                self.p_layers.add(nn.Dense(in_units=inp, units=out))

    def hybrid_forward(self, F, X):
        h = X
        for i, layer in enumerate(self.p_layers):
            h = layer(h)
            if i != len(self.p_layers) - 1:
                h = F.tanh(h)
        return h

In [8]:
class MultiVAE(HybridBlock):
    def __init__(
        self,
        p_dims: List[int],
        dropout_enc: List[float],
        dropout_dec: List[float],
        q_dims: List[int] = None,
    ):
        super().__init__()

        self.encode = VAEEncoder(q_dims, dropout_enc)
        self.decode = Decoder(p_dims, dropout_dec)

    def hybrid_forward(self, F, X):
        mu, logvar = self.encode(X)
        if autograd.is_training():
            std = F.exp(0.5 * logvar)
            eps = F.random.normal_like(std)
            mu = (eps * std) + mu
        return self.decode(mu), mu, logvar

...and, that's it...wait! We need the loss. Let me bring Eq 10 again

$$
\begin{equation*}
\mathcal L(\textbf{x}_u, \phi,\theta) = - \frac{1}{N_u}\sum_i x_{ui} \left[ \textit{log_softmax}(\textbf{z}_u) \right] + \beta \frac{1}{2N_u}\left[-\sum_i\left(\log\sigma_i^2 + 1\right) + \sum_i\sigma_i^2 + \sum_i\mu^2_i\right] \hspace{5cm} (10)
\end{equation*}
$$

In code

In [6]:
def vae_loss_fn(inp, out, mu, logvar, anneal):
    # firt term
    neg_ll = -nd.mean(nd.sum(nd.log_softmax(out) * inp, -1))
    # second term without beta
    KLD = -0.5 * nd.mean(nd.sum(1 + logvar - nd.power(mu, 2) - nd.exp(logvar), axis=1))
    # "full" loss (anneal is beta)
    return neg_ll + anneal * KLD

As I mentioned before, in the paper the authors also use a Multinomial Denoising Autoencoder (Mult-DAE}. The architecture is identical to that of the $\text{Mult-VAE}^{\text{PR}}$ apart from the fact that there is no variational aspect here. Therefore, not $D_{KL}$ regularization or reparameterization trick. The `Decoder` is identical to that shown above. The Encoder is as simple as:

In [9]:
class DAEEncoder(HybridBlock):
    def __init__(self, q_dims: List[int], dropout: List[float]):
        super().__init__()

        with self.name_scope():
            self.q_layers = nn.HybridSequential(prefix="q_net")
            for p, inp, out in zip(dropout, q_dims[:-1], q_dims[1:]):
                self.q_layers.add(nn.Dropout(p))
                self.q_layers.add(nn.Dense(in_units=inp, units=out))

    def hybrid_forward(self, F, X):
        h = F.L2Normalization(X)
        for layer in self.q_layers:
            h = F.tanh(layer(h))
        return h

and the model itself

In [10]:
class MultiDAE(HybridBlock):
    def __init__(
        self,
        p_dims: List[int],
        dropout_enc: List[float],
        dropout_dec: List[float],
        q_dims: List[int] = None,
    ):
        super().__init__()

        self.encode = DAEEncoder(q_dims, dropout_enc)
        self.decode = Decoder(p_dims, dropout_dec)

    def hybrid_forward(self, F, X):
        return self.decode(self.encode(X))

so, given the following parameters

In [23]:
I = 50000
q_dims = [I] + [600, 200]
p_dims = [200, 600] + [I]
dropout_enc = [0.5, 0.]
dropout_dec = [0., 0.]

In [24]:
vae_model = MultiVAE(
    p_dims=p_dims,
    q_dims=q_dims,
    dropout_enc=dropout_enc,
    dropout_dec=dropout_dec,
)

In [26]:
vae_model

MultiVAE(
  (encode): VAEEncoder(
    (q_layers): HybridSequential(
      (0): Dropout(p = 0.5, axes=())
      (1): Dense(50000 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 400, linear)
    )
  )
  (decode): Decoder(
    (p_layers): HybridSequential(
      (0): Dropout(p = 0.0, axes=())
      (1): Dense(200 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 50000, linear)
    )
  )
)

In [27]:
dae_model = MultiDAE(
    p_dims=p_dims,
    q_dims=q_dims,
    dropout_enc=dropout_enc,
    dropout_dec=dropout_dec,
)

In [28]:
dae_model

MultiDAE(
  (encode): DAEEncoder(
    (q_layers): HybridSequential(
      (0): Dropout(p = 0.5, axes=())
      (1): Dense(50000 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 200, linear)
    )
  )
  (decode): Decoder(
    (p_layers): HybridSequential(
      (0): Dropout(p = 0.0, axes=())
      (1): Dense(200 -> 600, linear)
      (2): Dropout(p = 0.0, axes=())
      (3): Dense(600 -> 50000, linear)
    )
  )
)

And with all this, we can now move to notebook 03 and see how we can train and evaluate the model